In [4]:
import os
import math
import random
import argparse
import copy
import ast
from collections import Counter, defaultdict
from itertools import chain, combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


In [5]:
from Nash import find_nash_equilibria_v2
from Functions import compute_and_visualize_shapley_values

In [6]:
# disable ConvergenceWarnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### C2

In [20]:
def corrupt_data(X: np.ndarray, corruption_prob: float = 0.8, nan_prob: float = 0.5,
                 noise_std: Optional[float] = None):
    if noise_std is None:
        noise_std = 0.1
    
    X = X.copy()
    n_samples, n_features = X.shape
    mask = np.random.rand(n_samples) < corruption_prob
    mask_nan = mask & (np.random.rand(n_samples) < nan_prob)
    mask_noise = mask & (~mask_nan)
    
    corrupt_features = range(n_features)

    # Apply corruption
    for feature in corrupt_features:
        X[mask_noise, feature] += np.random.randn(mask_noise.sum()) * noise_std

    X[np.argwhere(mask_nan)] = np.nan
    
    return X, mask_nan  # Return the `mask_nan` for label corruption

def corrupt_labels(y: np.ndarray, mask_nan: np.ndarray, corruption_prob: float = 0.2):
    y = y.copy()
    valid_mask = ~mask_nan  # Only consider rows not marked as NaN
    corruption_mask = (np.random.rand(len(y)) < corruption_prob) & valid_mask

    unique_labels = np.unique(y)

    # Generate corrupted labels ensuring no unchanged labels
    corrupted_labels = []
    for original_label in y[corruption_mask]:
        # Exclude the original label from the choices
        possible_labels = unique_labels[unique_labels != original_label]
        corrupted_label = np.random.choice(possible_labels)
        corrupted_labels.append(corrupted_label)
    
    y[corruption_mask] = corrupted_labels
    return y


def corrupt_and_save_clients_data(input_dir, output_dir, corrupt_client_indices, corruption_params, label_corruption_prob=0.2):
    os.makedirs(output_dir, exist_ok=True)

    for i in range(1, 11):  # Assuming 10 clients
        train_file = f"{input_dir}/train_{i:02d}.csv"
        df_train = pd.read_csv(train_file).dropna(subset=['act'])
        X_train = df_train.drop('act', axis=1).values
        y_train = df_train['act'].values

        test_file = f"{input_dir}/test_{i:02d}.csv"
        df_test = pd.read_csv(test_file).dropna(subset=['act'])
        X_test = df_test.drop('act', axis=1).values
        y_test = df_test['act'].values

        if i in corrupt_client_indices:
            # Corrupt features and get the mask of NaN rows
            X_train, mask_nan = corrupt_data(X_train, **corruption_params)
            # Corrupt labels, excluding rows marked as NaN
            y_train = corrupt_labels(y_train, mask_nan, corruption_prob=label_corruption_prob)

        pd.DataFrame(X_train).assign(act=y_train).to_csv(
            os.path.join(output_dir, f"train_{i:02d}.csv"), index=False
        )
        pd.DataFrame(X_test).assign(act=y_test).to_csv(
            os.path.join(output_dir, f"test_{i:02d}.csv"), index=False
        )

In [21]:
input_directory = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata"  # Path to input data
base_output_directory = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data"  # Base directory to save results
corruption_sets = [
    [1, 2, 3],
    [1, 2, 3, 4],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5, 6],
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8, 9]
]
corruption_params = {
    'corruption_prob': 0.3,
    'nan_prob': 0.5,
    'noise_std': 1
}

label_corruption_prob = 0.4  # Probability of corrupting labels

for corrupt_clients in corruption_sets:
    output_directory = os.path.join(base_output_directory, f"c{len(corrupt_clients)}")
    os.makedirs(output_directory, exist_ok=True)

    corrupt_and_save_clients_data(input_directory, output_directory, corrupt_clients, corruption_params, label_corruption_prob)
    print(f"Corrupted data for {len(corrupt_clients)} clients (clients {corrupt_clients}) saved in '{output_directory}'.")

Corrupted data for 3 clients (clients [1, 2, 3]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c3'.
Corrupted data for 4 clients (clients [1, 2, 3, 4]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c4'.
Corrupted data for 5 clients (clients [1, 2, 3, 4, 5]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c5'.
Corrupted data for 6 clients (clients [1, 2, 3, 4, 5, 6]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c6'.
Corrupted data for 8 clients (clients [1, 2, 3, 4, 5, 6, 7, 8]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c8'.
Corrupted data for 9 clients (clients [1, 2, 3, 4, 5, 6, 7, 8, 9]) saved in '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c9'.


# FedLR in HuGaDB with Different number of Corrupted Clients

In [37]:
def run_experiment(n_clients=10, n_trials=10, base_random_seed=42, subsample_ratios=[0.5, 0.6, 0.7, 0.8], max_iters=[10, 100], train_files_pattern='', test_files_pattern=''):
    all_results = []
    all_nash_equilibrium_counts = []
    all_nash_details = []

    for subsample_ratio in subsample_ratios:
        for max_iter in max_iters:
            print(f"\nRunning experiment with subsample_ratio = {subsample_ratio}, max_iter = {max_iter}")
            results = []
            nash_equilibrium_counts = Counter()
            nash_details_all = []

            for trial in range(n_trials):
                print(f"  Trial {trial + 1}/{n_trials} for subsample_ratio={subsample_ratio}, max_iter={max_iter}")
                trial_seed = base_random_seed + trial
                random.seed(trial_seed)
                np.random.seed(trial_seed)

                df_train_global = pd.concat([
                    pd.read_csv(train_files_pattern.format(i=i)) for i in range(1, 11)
                ]).dropna().sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True)
                X_train_global, y_train_global = df_train_global.drop('act', axis=1), df_train_global['act']

                df_test_global = pd.concat([
                    pd.read_csv(test_files_pattern.format(i=i)) for i in range(1, 11)
                ]).dropna().sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True)
                X_test_global, y_test_global = df_test_global.drop('act', axis=1), df_test_global['act']

                label_encoder = LabelEncoder()
                y_train_global = label_encoder.fit_transform(y_train_global)
                y_test_global = label_encoder.transform(y_test_global)

                scaler = StandardScaler()
                X_train_global_scaled = scaler.fit_transform(X_train_global)
                X_test_global_scaled = scaler.transform(X_test_global)

                def aggregate_lr_models(models):
                    aggregated_model = LogisticRegression()
                    aggregated_model.coef_ = np.mean([model.coef_ for model in models], axis=0) + np.random.normal(0, 0.05, models[0].coef_.shape)
                    aggregated_model.intercept_ = np.mean([model.intercept_ for model in models], axis=0)
                    aggregated_model.classes_ = models[0].classes_
                    return aggregated_model

                client_models = []
                client_global_accuracies = {}

                for i in range(n_clients):
                    df_train_i = pd.read_csv(train_files_pattern.format(i=i+1)).dropna(subset=['act']).sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True).dropna()
                    X_train, y_train = df_train_i.drop('act', axis=1), label_encoder.transform(df_train_i['act'])
                    model = LogisticRegression(random_state=np.random.randint(0, 100000), max_iter=max_iter)
                    model.fit(scaler.fit_transform(X_train), y_train)
                    client_models.append(model)
                    client_global_accuracies[i] = accuracy_score(y_test_global, model.predict(X_test_global_scaled))
                    
                trial_results = []
                n_combinations = 2 ** n_clients
                
                for i in tqdm(range(1, n_combinations), desc=f"    Evaluating coalitions for trial {trial+1}"):
                    bin_str = format(i, f'0{n_clients}b')
                    client_indices = [j for j in range(n_clients) if bin_str[n_clients - 1 - j] == '1']
                    included_models = [client_models[j] for j in client_indices]
                    if not included_models:
                        continue
                    
                    aggregated_model = aggregate_lr_models(included_models)
                    y_pred_global = aggregated_model.predict(X_test_global_scaled)
                    acc_global = accuracy_score(y_test_global, y_pred_global)
                    
                    row = {
                        'Trial': trial + 1,
                        'Combination': bin_str,
                        'Clients': [j + 1 for j in client_indices],
                        'Global Accuracy': acc_global
                    }
                    for j in range(n_clients):
                        row[f'Client {j + 1} Accuracy'] = client_global_accuracies[j]
                    trial_results.append(row)
                
                df_trial_results = pd.DataFrame(trial_results)
                df_trial_results.set_index('Combination', inplace=True)
                
                nash_df = find_nash_equilibria_v2(df_trial_results.reset_index())
                for combination in nash_df['Combination']:
                    nash_equilibrium_counts[combination] += 1
                
                shapley_df = compute_and_visualize_shapley_values(
                    df_trial_results.reset_index(), client_global_accuracies, n_clients=n_clients, plot=False, print_df=False
                )
                shapley_df['Trial'] = trial + 1
                
                for idx, nash_row in nash_df.iterrows():
                    combination_bin = nash_row['Combination']
                    client_indices = [j for j in range(n_clients) if combination_bin[n_clients - 1 - j] == '1']
                    clients_list = [j + 1 for j in client_indices]
                    nash_details_all.append({
                        'Trial': trial + 1,
                        'Combination': combination_bin,
                        'Clients': ",".join(map(str, clients_list)),
                        'Global Accuracy': nash_row['Global Accuracy'],
                        'Subsample Ratio': subsample_ratio,
                        'Max Iter': max_iter,
                    })
                
                results.extend(trial_results)
                
            all_results.append(pd.DataFrame(results))
            df_nash_counts = pd.DataFrame(nash_equilibrium_counts.items(), columns=['Nash Equilibrium', 'Occurrences'])
            df_nash_counts['Subsample Ratio'] = subsample_ratio
            df_nash_counts['Max Iter'] = max_iter
            all_nash_equilibrium_counts.append(df_nash_counts)
            
            df_nash_counts.to_csv(f"Nash_Equilibrium_Counts_HuGaLR_subsample_{subsample_ratio}_maxiter_{max_iter}.csv", index=False)
            all_nash_details.append(pd.DataFrame(nash_details_all))
    
    pd.concat(all_results, ignore_index=True).to_csv("Final_Results.csv", index=False)
    pd.concat(all_nash_equilibrium_counts, ignore_index=True).to_csv("Final_Nash_Equilibrium_Counts.csv", index=False)
    pd.concat(all_nash_details, ignore_index=True).to_csv("Nash_Equilibrium_Details_HuGaLR.csv", index=False)
    
    print("Process complete. Results saved.")


In [38]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c3/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c3/test_{i:02d}.csv" 
run_experiment(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.6, 0.7, 0.8],  max_iters=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)



Running experiment with subsample_ratio = 0.5, max_iter = 10
  Trial 1/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:03<00:00, 312.19it/s]


  Trial 2/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:03<00:00, 311.39it/s]


  Trial 3/10 for subsample_ratio=0.5, max_iter=10


...

  Trial 9/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:05<00:00, 195.03it/s]


  Trial 10/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:05<00:00, 200.04it/s]


Process complete. Results saved.


In [41]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c4/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c4/test_{i:02d}.csv" 
run_experiment(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.6, 0.7, 0.8],  max_iters=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)



Running experiment with subsample_ratio = 0.5, max_iter = 10
  Trial 1/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:03<00:00, 309.11it/s]


  Trial 2/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:03<00:00, 300.76it/s]


  Trial 3/10 for subsample_ratio=0.5, max_iter=10


...

  Trial 9/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:05<00:00, 194.28it/s]


  Trial 10/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:05<00:00, 199.68it/s]


Process complete. Results saved.


In [39]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c6/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c6/test_{i:02d}.csv" 
run_experiment(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.6, 0.7, 0.8],  max_iters=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_iter = 10
  Trial 1/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:03<00:00, 308.51it/s]


  Trial 2/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:03<00:00, 322.37it/s]


  Trial 3/10 for subsample_ratio=0.5, max_iter=10


...

  Trial 9/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:05<00:00, 197.24it/s]


  Trial 10/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:05<00:00, 196.95it/s]


Process complete. Results saved.


In [40]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c8/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c8/test_{i:02d}.csv" 
run_experiment(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.6, 0.7, 0.8],  max_iters=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)



Running experiment with subsample_ratio = 0.5, max_iter = 10
  Trial 1/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:03<00:00, 313.62it/s]


  Trial 2/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:03<00:00, 318.80it/s]


  Trial 3/10 for subsample_ratio=0.5, max_iter=10


...

  Trial 9/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:06<00:00, 165.28it/s]


  Trial 10/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:06<00:00, 167.64it/s]


Process complete. Results saved.


In [36]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c9/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c9/test_{i:02d}.csv" 
run_experiment(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.6, 0.7, 0.8],  max_iters=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)



Running experiment with subsample_ratio = 0.5, max_iter = 10
  Trial 1/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:03<00:00, 289.35it/s]


  Trial 2/10 for subsample_ratio=0.5, max_iter=10


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:03<00:00, 309.77it/s]


  Trial 3/10 for subsample_ratio=0.5, max_iter=10


...

  Trial 9/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:05<00:00, 193.50it/s]


  Trial 10/10 for subsample_ratio=0.8, max_iter=100


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:05<00:00, 193.50it/s]


Process complete. Results saved.


# FedFor in HuGaDB with Different number of Corrupted Clients

In [7]:
# Define the DecisionTree class
class DecisionTree:
    def __init__(self, max_depth=None, random_state=None):
        self.max_depth = max_depth
        self.tree_ = None
        self.random_state = random_state

    def fit(self, X, y):
        self.tree_ = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
        self.tree_.fit(X, y)

    def predict(self, X):
        return self.tree_.predict(X)

# Define the FederatedForest class
class FederatedForest:
    def __init__(self):
        self.models = []

    def add_model(self, model):
        self.models.append(model)

    def predict(self, X):
        predictions = [model.predict(X) for model in self.models]
        predictions = np.array(predictions).T
        y = [np.bincount(row).argmax() for row in predictions]
        return np.array(y)

In [8]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [9]:
def run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42, subsample_ratios=[0.6, 0.7, 0.8], max_depths=[10, 100],train_files_pattern='',test_files_pattern=''):
    all_results = []                   
    all_nash_equilibrium_counts = []   
    all_nash_details = []              
    
    for subsample_ratio in subsample_ratios:
        for max_depth in max_depths:
            print(f"\nRunning experiment with subsample_ratio = {subsample_ratio}, max_depth = {max_depth}")
            results = []
            nash_equilibrium_counts = Counter()
            nash_details_all = []  
    
            for trial in range(n_trials):
                print(f"  Trial {trial + 1}/{n_trials} for subsample_ratio={subsample_ratio}, max_depth={max_depth}")
                trial_seed = base_random_seed + trial
                random.seed(trial_seed)
                np.random.seed(trial_seed)
                
                df_train_global = pd.concat([pd.read_csv(train_files_pattern.format(i=i)) for i in range(1, 11)]).dropna()
                df_train_global = df_train_global.sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True)
                X_train_global = df_train_global.drop('act', axis=1)
                y_train_global = df_train_global['act']

                df_test_global = pd.concat([pd.read_csv(test_files_pattern.format(i=i)) for i in range(1, 11)]).dropna()
                df_test_global = df_test_global.sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True)
                X_test_global = df_test_global.drop('act', axis=1)
                y_test_global = df_test_global['act']

                label_encoder = LabelEncoder()
                y_train_global = label_encoder.fit_transform(y_train_global)
                y_test_global = label_encoder.transform(y_test_global)

                scaler = StandardScaler()
                X_train_global_scaled = scaler.fit_transform(X_train_global)
                X_test_global_scaled = scaler.transform(X_test_global)

                client_models = []
                client_global_accuracies = {}
                for i in range(n_clients):
                    df_train_i = pd.read_csv(train_files_pattern.format(i=i+1)).dropna(subset=['act'])
                    df_train_i = df_train_i.sample(frac=subsample_ratio, random_state=trial_seed).reset_index(drop=True)
                    X_train = df_train_i.drop('act', axis=1)
                    y_train = label_encoder.transform(df_train_i['act'])
                    
                    model = DecisionTree(max_depth=max_depth, random_state=np.random.randint(0, 100000))
                    model.fit(X_train, y_train)
                    client_models.append(model)
                    
                    y_pred_global = model.predict(X_test_global)
                    acc_global = accuracy_score(y_test_global, y_pred_global)
                    client_global_accuracies[i] = acc_global

                trial_results = []
                n_combinations = 2 ** n_clients
                for i in tqdm(range(1, n_combinations), desc=f" Evaluating coalitions for trial {trial+1}"):
                    bin_str = format(i, f'0{n_clients}b')
                    client_indices = [j for j in range(n_clients) if bin_str[n_clients - 1 - j] == '1']
                    included_models = [client_models[j] for j in client_indices]
                    if not included_models:
                        continue
                    
                    forest = FederatedForest()
                    for model in included_models:
                        forest.add_model(model)
                    
                    y_pred_global = forest.predict(X_test_global_scaled)
                    acc_global = accuracy_score(y_test_global, y_pred_global)
                    row = {'Trial': trial + 1, 'Combination': bin_str, 'Clients': [j + 1 for j in client_indices], 'Global Accuracy': acc_global}
                    for j in range(n_clients):
                        row[f'Client {j + 1} Accuracy'] = client_global_accuracies[j]
                    trial_results.append(row)
                
                df_trial_results = pd.DataFrame(trial_results)
                df_trial_results.set_index('Combination', inplace=True)
                nash_df = find_nash_equilibria_v2(df_trial_results.reset_index())
                for combination in nash_df['Combination']:
                    nash_equilibrium_counts[combination] += 1
                
                shapley_df = compute_and_visualize_shapley_values(df_trial_results.reset_index(), client_global_accuracies, n_clients=n_clients, plot=False, print_df = False)
                shapley_df['Trial'] = trial + 1

                for _, nash_row in nash_df.iterrows():
                    combination_bin = nash_row['Combination']
                    client_indices = [j for j in range(n_clients) if combination_bin[n_clients - 1 - j] == '1']
                    clients_list = [j + 1 for j in client_indices]
                    detail_record = {'Trial': trial + 1, 'Combination': combination_bin, 'Clients': ",".join(str(c) for c in clients_list)}
                    for client in range(1, n_clients + 1):
                        if client in clients_list:
                            detail_record[f'Client {client} Accuracy'] = client_global_accuracies[client - 1]
                            val = shapley_df.loc[shapley_df['Client'] == f'Client {client}', 'Normalized Shapley Value']
                            detail_record[f'Client {client} Shapley Value'] = val.values[0] if not val.empty else np.nan
                        else:
                            detail_record[f'Client {client} Accuracy'] = np.nan
                            detail_record[f'Client {client} Shapley Value'] = np.nan
                    detail_record['Global Accuracy'] = nash_row['Global Accuracy']
                    detail_record['Subsample Ratio'] = subsample_ratio
                    detail_record['Max Depth'] = max_depth
                    nash_details_all.append(detail_record)
                results.extend(trial_results)

            df_results = pd.DataFrame(results)
            df_results['Subsample Ratio'] = subsample_ratio
            df_results['Max Depth'] = max_depth
            all_results.append(df_results)

            df_nash_counts = pd.DataFrame(nash_equilibrium_counts.items(), columns=['Nash Equilibrium', 'Occurrences'])
            df_nash_counts['Subsample Ratio'] = subsample_ratio
            df_nash_counts['Max Depth'] = max_depth
            all_nash_equilibrium_counts.append(df_nash_counts)
            df_nash_counts.to_csv(f"Nash_Equilibrium_Counts_HuGaFedFor_subsample_{subsample_ratio}_max_depth_{max_depth}.csv", index=False)

            df_nash_details = pd.DataFrame(nash_details_all)
            all_nash_details.append(df_nash_details)
    
    pd.concat(all_nash_equilibrium_counts, ignore_index=True).to_csv("Nash_Equilibrium_Counts.csv", index=False)
    pd.concat(all_nash_details, ignore_index=True).to_csv("Nash_Equilibrium_Details_HuGaFedFor.csv", index=False)
    print("Process complete. Results saved.")


In [10]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c3/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c3/test_{i:02d}.csv" 
run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.8],  max_depths=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_depth = 10
  Trial 1/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:47<00:00, 21.32it/s]


  Trial 2/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:49<00:00, 20.77it/s]


  Trial 3/10 for subsample_ratio=0.5, max_depth=10


...

  Trial 9/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [01:23<00:00, 12.32it/s]


  Trial 10/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [18:32<00:00,  1.09s/it]   

Process complete. Results saved.


In [10]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c4/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c4/test_{i:02d}.csv" 
run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.8],  max_depths=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_depth = 10
  Trial 1/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:48<00:00, 21.07it/s]


  Trial 2/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:48<00:00, 21.23it/s]


  Trial 3/10 for subsample_ratio=0.5, max_depth=10


...

  Trial 9/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [01:25<00:00, 11.98it/s]


  Trial 10/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [01:24<00:00, 12.16it/s]

Process complete. Results saved.


In [11]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c6/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c6/test_{i:02d}.csv" 
run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.8],  max_depths=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_depth = 10
  Trial 1/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:49<00:00, 20.50it/s]


  Trial 2/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:47<00:00, 21.41it/s]


  Trial 3/10 for subsample_ratio=0.5, max_depth=10


...

  Trial 9/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [01:27<00:00, 11.74it/s]


  Trial 10/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [01:28<00:00, 11.60it/s]

Process complete. Results saved.


In [12]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c8/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c8/test_{i:02d}.csv" 
run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.8],  max_depths=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_depth = 10
  Trial 1/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:48<00:00, 21.31it/s]


  Trial 2/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:47<00:00, 21.34it/s]


  Trial 3/10 for subsample_ratio=0.5, max_depth=10


...

  Trial 9/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [01:28<00:00, 11.60it/s]


  Trial 10/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [01:28<00:00, 11.51it/s]

Process complete. Results saved.


In [13]:
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c9/train_{i:02d}.csv" 
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/corrupted_data/c9/test_{i:02d}.csv" 
run_experiment_2(n_clients=10, n_trials=10, base_random_seed=42,  subsample_ratios=[0.5, 0.8],  max_depths=[10, 100], train_files_pattern=train_files_pattern, test_files_pattern=test_files_pattern)


Running experiment with subsample_ratio = 0.5, max_depth = 10
  Trial 1/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:48<00:00, 21.10it/s]


  Trial 2/10 for subsample_ratio=0.5, max_depth=10


 Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:49<00:00, 20.85it/s]


  Trial 3/10 for subsample_ratio=0.5, max_depth=10


...

  Trial 9/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [01:38<00:00, 10.40it/s]


  Trial 10/10 for subsample_ratio=0.8, max_depth=100


 Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [01:38<00:00, 10.38it/s]


Process complete. Results saved.
